# Imports

In [29]:
import itertools
import networkx as nx
import numpy as np

# Config

In [2]:
# board size = length of one side of hexagonal board
board_size = 4

# Create Board

* hexagonal 2d space can be represented with 3 axes on a 2d plane equally 
separated
* each integer coordinate is a single square on the board
* neighbours are then +1 in each axes, maxed at the size of the board

In [81]:
nodes = np.array(
    list(itertools.product(*[range(-board_size + 1, board_size)] * 2))
)
directions = list(itertools.product(*[[-1, 0, 1]] * 2))
directions.pop(directions.index((0, 0)))
directions = np.array(directions)

In [82]:
edges = {}
for i, node in enumerate(nodes):
    edges[i] = []
    for direction in directions:
        neighbour = node + direction
        if (np.abs(neighbour) >= board_size).sum() > 0:
            continue
        neighbour_index = nodes.tolist().index(neighbour.tolist())
        edges[i].append(neighbour_index)
g = nx.from_dict_of_lists(edges)

In [88]:
print(f"from node:\n{nodes[0]}\n")
print("to nodes:")
for to_node in g[0].keys():
    print(nodes[to_node])

from node:
[-3 -3]

to nodes:
[-3 -2]
[-2 -3]
[-2 -2]


In [77]:
nodes[0]

array([-3, -3, -3])

In [78]:
4 + 5 + 6 + 7 + 6 + 5 + 4

37

In [84]:
len(nodes)

49